In [1]:
import glob
import os
import tables
import astropy.units as u
import hi_processing.images as hip
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import simplejson as json
from sklearn.neighbors import KernelDensity
from skimage import measure
import SolarStormwatchIIAnalysis as ssw2
import moviepy.editor as mpy
import palettable.colorbrewer.qualitative as cmap

%matplotlib inline

colors = cmap.Paired_12.mpl_colors
mksz = 8
# Get some colors to use for the different time-elongation profile sources and speed estimate methods.
# t-e profile sources:


C:\Users\Luke\Anaconda2\lib\site-packages\glymur\lib\config.py:151: UserWarning: Neither the openjp2 nor the openjpeg library could be loaded.  
  warnings.warn(msg)
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '



In [4]:
def make_ogv(src, dst, tidy=True):
    file_list = glob.glob(src)
    clip = mpy.ImageSequenceClip(file_list, fps=1)
    clip.write_videofile(dst, audio=False, ffmpeg_params=['-qscale','6'])
    if tidy:
        for f in file_list:
            os.remove(f)
            
def get_fwhm(x, pdf):
    """
    Function to calculate the full width at half maximum of a 1-d distribution.
    :param x:
    :param pdf:
    :return:
    """

    #Calc FWHM
    pdf_max = pdf.max()
    pdf_hm = pdf_max/2.0
    id_max = np.argwhere(pdf==pdf_max).ravel()
    if len(id_max)>1:
        id_max = id_max[0]
    x_max = x[id_max]

    # Look low.
    pdf_l = pdf[x<x_max]
    x_l = x[x<x_max]
    if len(x_l)>1:
        x_hm_l = x_l[np.argmin(np.abs(pdf_l - pdf_hm))]
    else:
        x_hm_l = np.NaN

    # Look high.
    pdf_h = pdf[x>x_max]
    x_h = x[x>x_max]
    if len(x_h)>1:
        x_hm_h = x_h[np.argmin(np.abs(pdf_h - pdf_hm))]
    else:
        x_hm_h = np.NaN

    return x_hm_l, x_max, x_hm_h


def extract_cme_front_example(pix_coords, hpr_coords, hi_map, kernel="epanechnikov", bandwidth=40, thresh=10):
    """
    A function that uses Kernel density estimation and skeletonization to estimate the CME front location from the
    cloud of classifications for this asset.
    :param pix_coords: Dataframe of classification coordinates, with columns 'x', and 'y' for the x pixels and y pixels.
    :param hpr_coords: Dataframe of classification coordinates, with columns 'pa', and 'el' for the pixel coords in HPR.
    :param hi_map: A sunpy map of the HI image current being analysed.
    :param kernel: String name of any valid kernal in scikit-learn.neighbours.KernalDensity.
    :param bandwidth: Float value of the bandwidth to be used by the kernal. Defaults to 40 pixels, as this is
                      approximately the error in classifications if made using a touchscreen.
    :param thresh: Float value of threshold to apply to the density map to identify the CME skeleton. Defaults to 10,
                   but this is only from playing around and hasn't been thoroughly testted.
    :return: coords: A dataframe with columns:
                     x: x-pixel coords of the best estimate of the cme front
                     y: y-pixel coords of the best estimate of the cme front
                     pa:
                     el:
                     el_lo:
                     el_hi:
    """
    if pix_coords.shape[0] < 20:
        print("Error: <20 coordinates provided, CME front identification likely to be poor")

    if kernel not in {"gaussian", "tophat", "epanechnikov", "exponential", "linear", "cosine"}:
        print("Error: invalid kernel, defaulting to epanechnikov")
        kernel = "epanechnikov"

    if not isinstance(bandwidth, (int, float)):
        print("Error: bandwidth should be an int or float number. defaulting to 40")
        bandwidth = 40

    if not isinstance(thresh, (int, float)):
        print("Error: thresh should be an int or float number. defaulting to 10")
        bandwidth = 10

    # Get all pixel coordinates in the image frame.
    x = np.arange(0, 1024)
    y = np.arange(0, 1024)
    xm, ym = np.meshgrid(x, y)
    # Get all coords for KSdensity
    all_coords = np.vstack([xm.ravel(), ym.ravel()]).T
    # Lookup HPR coords in frame
    elm, pam = hip.convert_pix_to_hpr(xm * u.pix, ym * u.pix, hi_map)
    # Loose the units as not needed
    elm = elm.value
    pam = pam.value

    # Parse classification cloud to kernel density estimator. Returns log of density.
    xy = np.vstack([pix_coords['x'].values.ravel(), pix_coords['y'].values.ravel()]).T
    # Get flags for aborting the CME fit distributions
    fit_distribution = True
    fit_cme_distribution = True

    if xy.shape[0] == 0:
        fit_distribution = False

    if fit_distribution:
        # Use first stab at kernel density to
        kde = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(xy)
        log_pdf = np.reshape(kde.score_samples(all_coords), xm.shape)
        # Convert the pdf to a binary map of where the distribution is > np.max(dist)/thresh
        pdf = np.exp(log_pdf.copy())
        
        thresh = np.max(pdf) / thresh
        pdf_binary = pdf.copy()
        pdf_binary[pdf_binary < thresh] = 0
        pdf_binary[pdf_binary >= thresh] = 1
        # Only keep the largest CME feature in the binary map.
        # Label each binary blob
        pdf_binary_label = measure.label(pdf_binary.astype(int))
        # update label so that the background pixels have the same label of zero
        pdf_binary_label[pdf_binary == 0] = 0

        # Find how many unique labels and how many pixels in each label
        lab, cnt = np.unique(pdf_binary_label, return_counts=True)
        # Sort these so largest is last in list.
        # Largest should always be the background. So keep two largest - background and clearest CME identification
        lab = lab[np.argsort(cnt)]
        for l in lab[:-2]:
            pdf_binary_label[pdf_binary_label == l] = 0

        # Set the largest blob back to 1 rather than it's label.
        pdf_binary_label[pdf_binary_label != 0] = 1

        # Find classifications inside this polygon
        id_keep = []
        for idr, row in pix_coords.iterrows():
            good_row = True
            if (row['y'] < 0) | (row['y'] > 1023):
                print "row: {0}, y : {1}".format(idr, row['y'])
                good_row = False
            if (row['x'] < 0) | (row['x'] > 1023):
                print "row: {0}, x : {1}".format(idr, row['x'])
                good_row = False

            if good_row:
                if pdf_binary_label[int(row['y']), int(row['x'])] == 1:
                    id_keep.append(True)
                else:
                    id_keep.append(False)
            else:
                id_keep.append(False)

        xy_in_cme = np.vstack([pix_coords['x'][id_keep].values.ravel(), pix_coords['y'][id_keep].values.ravel()]).T
        hpr_in_cme = np.vstack([hpr_coords['pa'][id_keep].values.ravel(), hpr_coords['el'][id_keep].values.ravel()]).T

        # Now fit only these points
        # Are there any points left?
        if xy_in_cme.shape[0] == 0:
            fit_cme_distribution = False

        if fit_cme_distribution:
            kde = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(xy_in_cme)
            log_pdf = np.reshape(kde.score_samples(all_coords), xm.shape)
            cme_pdf = np.exp(log_pdf.copy())

        # Use classifications to find range of PA values to look up CME elongation for.
        # Preallocate space for CME front.
        # Use ceil and floor so that pa_vals definitely stays in range.
        # pa_vals = np.arange(np.ceil(hpr_in_cme[:,0].min()), np.floor(hpr_in_cme[:,0].max()), 1)
        # cme_el_peak = np.zeros(pa_vals.shape)
        # cme_el_lo = np.zeros(pa_vals.shape)
        # cme_el_hi = np.zeros(pa_vals.shape)
            # Loop over position angles, get FWHM of the kernel density estimate along constant PA.
            # Use FWHM as elon error est.
        #  for i, pa in enumerate(pa_vals):
        #      # Get pixel coords of this PA slice
        #      pa_slice = measure.find_contours(pam,pa)[0]
        #      # Get values rounded to nearest pixel
        #      pa_slice = pa_slice.astype(int)
        #      pdf_pa = cme_pdf[pa_slice[:, 0], pa_slice[:, 1]]
        #      el_pa = elm[pa_slice[:, 0], pa_slice[:, 1]]
        #      cme_el_lo[i], cme_el_peak[i], cme_el_hi[i] = get_fwhm(el_pa, pdf_pa)

    return pdf, pdf_binary, pdf_binary_label, xy_in_cme, cme_pdf, pam, elm

In [37]:
"""
A function to load in the test version of the HDF5 stormwatch data and produce a test plot.
:return:
"""

project_dirs = ssw2.get_project_dirs()
# Get the swpc cme database
swpc_cmes = ssw2.load_swpc_events()

# Import the SSW classifications data
ssw_out_name = os.path.join(project_dirs['out_data'], 'all_classifications_matched_ssw_events_plus_HPR.hdf5')
ssw_out = tables.open_file(ssw_out_name, mode="r")

# Make a directory to store all of these figures in
fig_out_dir = os.path.join(project_dirs['figs'], "process_animation")
if not os.path.exists(fig_out_dir):
    os.mkdir(fig_out_dir)

# Only do STA, EVENT 8, Differenced.

# Iterate through the events, and annotate storm position on each plot.
frame = 0

def add_annotation(ax, label, box_col='k', txt_x=0.008, txt_y=0.975, txt_fs=18):
    box = {'facecolor': box_col}
    txt = ax.text(txt_x, txt_y, label,
                  transform=ax.transAxes, fontsize=txt_fs, color='r', bbox=box)
    return txt

def save_frame(fig, frame, num_frames=1):
    for i in range(num_frames):
        out_name = "frame_{:02d}.jpg".format(frame)
        out_path = os.path.join(fig_out_dir, out_name)
        fig.subplots_adjust(left=0.01, bottom=0.01, right=0.99, top=0.99, wspace=0.02, hspace=0.02)
        fig.savefig(out_path)
        frame +=1
        
    return frame

for event in ssw_out.iter_nodes('/'):

    # Get ssw event number to loop up event HI start and stop time.
    ssw_num = int(event._v_name.split('_')[1])

    if ssw_num != 8:
        continue

    print ssw_num

    for craft in event:


        if craft._v_name != 'sta':
            continue

        print craft._v_name


        t_start = swpc_cmes.loc[ssw_num, 't_hi1a_start']
        t_stop = swpc_cmes.loc[ssw_num, 't_hi1a_stop']

        hi_files = hip.find_hi_files(t_start, t_stop, craft=craft._v_name, camera='hi1', background_type=1)

        fc = hi_files[1]
        fp = hi_files[0]

        for img_type in craft:

            if img_type._v_name != 'diff':
                continue

            hi_map = hip.get_image_diff(fc, fp, align=True, smoothing=True)
            normalise = mpl.colors.Normalize(vmin=-0.05, vmax=0.05)
            img = mpl.cm.gray(normalise(hi_map.data), bytes=True)
            
            # Plot out the raw frame
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.imshow(img, origin='lower')
            ax.set_xlim(0, 1023)
            ax.set_ylim(0, 1023)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            txt = add_annotation(ax, "STEREO-A HI1 Differenced image")
            frame = save_frame(fig, frame, num_frames=3)
            
            
            fc_base = os.path.basename(fc)
            time_label = "T"+ "_".join(fc_base.split('_')[0:2])
            
            # Get the time node for this file.
            time = img_type._f_get_child(time_label)
            
            # Loop over users and add them on.
            i=0
            for user in time.users:
                if i > len(colors)-1:
                    i = 0
                    
                user_data = pd.DataFrame.from_records(user.coords.read())
                user_data.replace(to_replace=[99999], value=np.NaN, inplace=True)
                # Add on all classifications
                ax.plot(user_data['x'], user_data['y'], 'o', markersize=mksz, color=colors[i])
                # Update label
                txt.set_text("Adding each citizen scientist classification")
                frame = save_frame(fig, frame)
                i += 1
                
            # Overlay all the data           
            all_data = pd.DataFrame.from_records(time.coords.read())
            all_data.replace(to_replace=[99999], value=np.NaN, inplace=True)
            # Add on all classifications as same color and save
            ax.plot(all_data['x'], all_data['y'], 'o', markersize=mksz, color='y')
            txt.set_text("All citizen scientist classifications")
            frame = save_frame(fig, frame, num_frames=3)
                                  
            # Now fit the kernal density distribution to this.
            pix_coords = pd.DataFrame({'x': all_data['x'], 'y': all_data['y']})
            hpr_coords = pd.DataFrame({'el': all_data['el'], 'pa': all_data['pa']})
            pdf, pdf_binary, pdf_binary_label, xy_in_cme, cme_pdf, pam, elm = extract_cme_front_example(pix_coords, hpr_coords, hi_map, kernel="epanechnikov", bandwidth=40, thresh=10)
            
            # Plot on the Kernel density estimate, and then the thresholded and segmented distributions
            # Raw densities
            ax.cla()
            ax.imshow(pdf,cmap=mpl.cm.viridis, origin="lower")
            
            ax.set_xlim(0, 1023)
            ax.set_ylim(0, 1023)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            add_annotation(ax,"Density estimate of all classifications")
            frame = save_frame(fig, frame, num_frames=3)
            
            # Thresholded densities
            ax.cla()
            ax.imshow(pdf_binary, cmap='gray', origin="lower")
            ax.set_xlim(0, 1023)
            ax.set_ylim(0, 1023)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            add_annotation(ax,"Thresholded density identifies largest features")
            frame = save_frame(fig, frame, num_frames=3)
            
            # Segmented densities
            ax.cla()
            ax.imshow(pdf_binary_label, cmap='gray', origin="lower")
            ax.set_xlim(0, 1023)
            ax.set_ylim(0, 1023)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            add_annotation(ax,"Segmentation extracts the CME region")
            frame = save_frame(fig, frame, num_frames=3)
            
            # Remaining points
            ax.cla()
            ax.imshow(pdf_binary_label, cmap='gray', origin="lower")
            ax.plot(xy_in_cme[:,0], xy_in_cme[:,1], 'o', color='y', markersize=mksz)
            ax.set_xlim(0, 1023)
            ax.set_ylim(0, 1023)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            add_annotation(ax,"Classifications within the CME region")
            frame = save_frame(fig, frame, num_frames=3)
            
            # CME distribution
            ax.cla()
            ax.imshow(cme_pdf, cmap=mpl.cm.viridis, origin="lower")
            ax.set_xlim(0, 1023)
            ax.set_ylim(0, 1023)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            add_annotation(ax, "Density estimate of classifications in CME region")
            frame = save_frame(fig, frame, num_frames=3)
            
            plt.close('all')
            
            fig, ax = plt.subplots(2,2,figsize=(10,10))
            ax = ax.flatten()
            ax_hi = ax[1]
            ax_sub = [ax[0],ax[2],ax[3]]
            ax_hi.imshow(cme_pdf, cmap=mpl.cm.viridis, origin="lower")
            ax_hi.set_xlim(0, 1023)
            ax_hi.set_ylim(0, 1023)
            ax_hi.get_xaxis().set_visible(False)
            ax_hi.get_yaxis().set_visible(False)
            txt = add_annotation(ax_hi,"Calculate CME location along lines of fixed PA", txt_x=-0.5, txt_y=0.975, txt_fs=18)

            for a in ax_sub:
                a.set_visible(False)

            frame = save_frame(fig, frame, num_frames=3)
            # Remove the annotation to make space for axis.
            txt.remove()
            
            # Add on PA contour.
            # Use classifications to find range of PA values to look up CME elongation for.
            pa_vals = np.array([60,90,120])
            cme_el_peak = np.zeros(pa_vals.shape)
            cme_el_lo = np.zeros(pa_vals.shape)
            cme_el_hi = np.zeros(pa_vals.shape)
            # Loop over position angles, get FWHM of the kernel density estimate along constant PA.
            for pa, a, col in zip(pa_vals, ax_sub, colors[3:8:2]):
                # Get pixel coords of this PA slice
                pa_slice = measure.find_contours(pam,pa)[0]
                # Add on the PA contour
                ax_hi.plot(pa_slice[:,1], pa_slice[:,0], '-', color=col, linewidth=3)

                # Get values rounded to nearest pixel
                pa_slice = pa_slice.astype(int)

                pdf_pa = cme_pdf[pa_slice[:, 0], pa_slice[:, 1]]
                el_pa = elm[pa_slice[:, 0], pa_slice[:, 1]]

                a.set_visible(True)
                a.plot(el_pa, pdf_pa,'-', color=col, linewidth=2)
                a.set_xlabel('Elongation (degrees)', fontsize=14)
                a.xaxis.set_label_coords(0.25, 0.075, transform=a.transAxes)
                a.set_ylabel('Classification density', fontsize=14)
                a.yaxis.set_label_coords(0.075, 0.5, transform=a.transAxes)

                cme_el_lo, cme_el_peak, cme_el_hi = get_fwhm(el_pa, pdf_pa)
                ymin, ymax = a.get_ylim()
                a.vlines(cme_el_peak, ymin, ymax, linestyles=['-'], colors=['k'], linewidths=[2])
                a.vlines([cme_el_lo, cme_el_hi], ymin, ymax, linestyles=['--'], colors=['k'], linewidths=[2])
                a.set_xlim(5,20)

                a.set_xticklabels([])
                a.set_yticklabels([])
                a.invert_xaxis()
                frame = save_frame(fig, frame, num_frames=3)
            plt.close('all')
            
            # Add on the CME front the the contour plot, then replace with the HI image
            fig, ax = plt.subplots(figsize=(10,10))
            ax.imshow(cme_pdf, cmap=mpl.cm.viridis, origin="lower")
            ax.set_xlim(0, 1023)
            ax.set_ylim(0, 1023)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            txt = add_annotation(ax,"Repeat for all PA")
            frame = save_frame(fig, frame, num_frames=3)
            txt.remove()
            
            # Add on all the position angle contours
            levels = np.arange(np.int(pam.min()),np.int(pam.max()),1)
            cnts = ax.contour(pam,levels=levels, colors=['r'], linewidths=[2])
            frame = save_frame(fig, frame, num_frames=3)
            
            # Remove the contour lines, replace with CME front
            for line in cnts.collections:
                line.remove()
            
            
            # Get the CME front.
            cme = pd.DataFrame.from_records(time.cme_coords.read())
            cme.replace(to_replace=[99999], value=np.NaN, inplace=True)
            # Add on CME front and error
            ax.plot(cme['x'], cme['y'], '-', color='r', linewidth=3 )
            ax.plot(cme['x_lo'], cme['y_lo'], '--', color='r', linewidth=3)
            ax.plot(cme['x_hi'], cme['y_hi'], '--', color='r', linewidth=3)
            add_annotation(ax,"The estimate of the CME front")
            frame = save_frame(fig, frame, num_frames=3)
            
            ax.cla()
            ax.imshow(img, origin='lower')
            ax.set_xlim(0, 1023)
            ax.set_ylim(0, 1023)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            
            
            # Get the CME front.
            cme = pd.DataFrame.from_records(time.cme_coords.read())
            cme.replace(to_replace=[99999], value=np.NaN, inplace=True)
            # Add on CME front and error
            ax.plot(cme['x'], cme['y'], '-', color='r', linewidth=3 )
            ax.plot(cme['x_lo'], cme['y_lo'], '--', color='r', linewidth=3)
            ax.plot(cme['x_hi'], cme['y_hi'], '--', color='r', linewidth=3)
            add_annotation(ax,"The estimate of the CME front")
            frame = save_frame(fig, frame, num_frames=3)
            
            # Now Loop through the differenced images to show how the tracking works.
            files_current = hi_files[2:10]
            files_previous = hi_files[1:9]
            plt.close('all')
            
            for fc, fp in zip(files_current, files_previous):

                hi_map = hip.get_image_diff(fc, fp, align=True, smoothing=True)
                normalise = mpl.colors.Normalize(vmin=-0.05, vmax=0.05)
                img = mpl.cm.gray(normalise(hi_map.data), bytes=True)

                # Plot out the raw frame
                fig, ax = plt.subplots(figsize=(10, 10))
                ax.imshow(img, origin='lower')
                ax.set_xlim(0, 1023)
                ax.set_ylim(0, 1023)
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)

                fc_base = os.path.basename(fc)
                time_label = "T"+ "_".join(fc_base.split('_')[0:2])

                # Get the time node for this file.
                time = img_type._f_get_child(time_label)
                
                cme = pd.DataFrame.from_records(time.cme_coords.read())
                cme.replace(to_replace=[99999], value=np.NaN, inplace=True)
                # Add on CME front and error
                ax.plot(cme['x'], cme['y'], '-', color='r', linewidth=3 )
                ax.plot(cme['x_lo'], cme['y_lo'], '--', color='r', linewidth=3)
                ax.plot(cme['x_hi'], cme['y_hi'], '--', color='r', linewidth=3)
                add_annotation(ax,"Repeat for later frames")
                frame = save_frame(fig, frame)
                plt.close('all')


src = os.path.join(fig_out_dir, '*.jpg')            
dst = os.path.join(fig_out_dir, 'ssw_processing_illustration.ogv')
make_ogv(src, dst, tidy=False)

Error, invalid path, check config: F:\STEREO\ares.nrl.navy.mil\lz
Error, invalid path, check config: F:\STEREO\ares.nrl.navy.mil\lz
8
sta
[MoviePy] >>>> Building video C:\Users\Luke\PycharmProjects\SolarStormwatchIIAnalysis\figures\process_animation\ssw_processing_illustration.ogv
[MoviePy] Writing video C:\Users\Luke\PycharmProjects\SolarStormwatchIIAnalysis\figures\process_animation\ssw_processing_illustration.ogv


 99%|█████████████████████████████████████████▍| 70/71 [00:04<00:00, 15.67it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: C:\Users\Luke\PycharmProjects\SolarStormwatchIIAnalysis\figures\process_animation\ssw_processing_illustration.ogv 

